<a href="https://colab.research.google.com/github/aguilarmadeira/ColabTest/blob/main/linhas_nurullah1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy.optimize import minimize_scalar

# Função Rastrigin em 10 dimensões
def rastrigin(x):
    A = 10
    return A * len(x) + sum([(xi**2 - A * np.cos(2 * np.pi * xi)) for xi in x])

# Cosdense curve para redução dimensional
def cosdense(z, l, u, alpha):
    n = len(l)
    beta = [1] + [(np.pi/alpha)**i * np.prod([u[j]-l[j] for j in range(i)]) for i in range(1,n)]
    return [(l[i]-u[i])/2 * np.cos(beta[i]*z) + (l[i]+u[i])/2 for i in range(n)]

bounds = [-5.12]*10, [5.12]*10
alpha = 0.01

# Caso 1: 10D->1D
def f1D(z):
    x = cosdense(z, bounds[0], bounds[1], alpha)
    return rastrigin(x)

result1D = minimize_scalar(f1D, bounds=(0,np.pi), method='bounded')

# Caso 2: 10D->2D->1D
def f2D(z1,z2):
    x1 = cosdense(z1, bounds[0][:5], bounds[1][:5], alpha)
    x2 = cosdense(z2, bounds[0][5:], bounds[1][5:], alpha)
    return rastrigin(x1 + x2)

def f2D_1D(t):
    z1 = (np.pi/2)*(1-np.cos(t))
    z2 = (np.pi/2)*(1-np.cos(1.5*t))
    return f2D(z1,z2)

result2D1D = minimize_scalar(f2D_1D, bounds=(0,np.pi), method='bounded')

# Caso 3: 10D->5D->2D->1D
# Redução intermédia para 5D (dois grupos de 5 variáveis)
def f5D(z):
    x = []
    for i in range(2):
        x += cosdense(z[i], bounds[0][i*5:(i+1)*5], bounds[1][i*5:(i+1)*5], alpha)
    return rastrigin(x)

# Redução para 2D
def f2D_from_5D(u,v):
    z = [(np.pi/2)*(1-np.cos(u)), (np.pi/2)*(1-np.cos(v))]
    return f5D(z)

# Redução final para 1D
def f1D_final(t):
    u = (np.pi/2)*(1-np.cos(t))
    v = (np.pi/2)*(1-np.cos(1.5*t))
    return f2D_from_5D(u,v)

result5D2D1D = minimize_scalar(f1D_final, bounds=(0,np.pi), method='bounded')

# Resultados finais
print('Resultado 10D->1D:', result1D.fun)
print('Resultado 10D->2D->1D:', result2D1D.fun)
print('Resultado 10D->5D->2D->1D:', result5D2D1D.fun)

Resultado 10D->1D: 154.74281353643283
Resultado 10D->2D->1D: 156.02621688062192
Resultado 10D->5D->2D->1D: 152.82046016849455


In [2]:
import numpy as np
from scipy.optimize import golden

# Função Rastrigin em 10 dimensões
def rastrigin(x):
    A = 10
    return A * len(x) + sum([(xi**2 - A * np.cos(2 * np.pi * xi)) for xi in x])

# Curva Cosdense
def cosdense(z, l, u, alpha):
    n = len(l)
    beta = [1] + [(np.pi/alpha)**i * np.prod([u[j]-l[j] for j in range(i)]) for i in range(1,n)]
    return [(l[i]-u[i])/2 * np.cos(beta[i]*z) + (l[i]+u[i])/2 for i in range(n)]

bounds = [-5.12]*10, [5.12]*10
alpha = 0.01

# Contador de avaliações
def count_eval(f):
    def wrapper(x):
        wrapper.count += 1
        return f(x)
    wrapper.count = 0
    return wrapper

# Caso 1: 10D->1D
def f1D(z):
    x = cosdense(z, bounds[0], bounds[1], alpha)
    return rastrigin(x)

f1D_counted = count_eval(f1D)
res1 = golden(f1D_counted, brack=(0, np.pi), full_output=True)

# Caso 2: 10D->2D->1D
def f2D(z1,z2):
    x1 = cosdense(z1, bounds[0][:5], bounds[1][:5], alpha)
    x2 = cosdense(z2, bounds[0][5:], bounds[1][5:], alpha)
    return rastrigin(x1 + x2)

def f2D_1D(t):
    z1 = (np.pi/2)*(1-np.cos(t))
    z2 = (np.pi/2)*(1-np.cos(1.5*t))
    return f2D(z1,z2)

f2D_1D_counted = count_eval(f2D_1D)
res2 = golden(f2D_1D_counted, brack=(0, np.pi), full_output=True)

# Caso 3: 10D->5D->2D->1D
def f5D(z):
    x = []
    for i in range(2):
        x += cosdense(z[i], bounds[0][i*5:(i+1)*5], bounds[1][i*5:(i+1)*5], alpha)
    return rastrigin(x)

def f2D_from_5D(u,v):
    z = [(np.pi/2)*(1-np.cos(u)), (np.pi/2)*(1-np.cos(v))]
    return f5D(z)

def f1D_final(t):
    u = (np.pi/2)*(1-np.cos(t))
    v = (np.pi/2)*(1-np.cos(1.5*t))
    return f2D_from_5D(u,v)

f1D_final_counted = count_eval(f1D_final)
res3 = golden(f1D_final_counted, brack=(0, np.pi), full_output=True)

# Resultados finais
print('\nResultados Método da Secção Áurea:\n')
print(f'Caso 10D->1D: resultado={res1[0]:.5f}, valor={res1[1]:.5f}, avaliações={f1D_counted.count}')
print(f'Caso 10D->2D->1D: resultado={res2[0]:.5f}, valor={res2[1]:.5f}, avaliações={f2D_1D_counted.count}')
print(f'Caso 10D->5D->2D->1D: resultado={res3[0]:.5f}, valor={res3[1]:.5f}, avaliações={f1D_final_counted.count}')


Resultados Método da Secção Áurea:

Caso 10D->1D: resultado=11.36639, valor=121.17793, avaliações=46
Caso 10D->2D->1D: resultado=4.34157, valor=153.98446, avaliações=43
Caso 10D->5D->2D->1D: resultado=2.69903, valor=161.76506, avaliações=45


In [3]:
import numpy as np
from scipy.optimize import direct

# Função Rastrigin em 10 dimensões
def rastrigin(x):
    A = 10
    return A * len(x) + sum([(xi**2 - A * np.cos(2 * np.pi * xi)) for xi in x])

# Curva Cosdense
def cosdense(z, l, u, alpha):
    n = len(l)
    beta = [1] + [(np.pi/alpha)**i * np.prod([u[j]-l[j] for j in range(i)]) for i in range(1,n)]
    return [(l[i]-u[i])/2 * np.cos(beta[i]*z) + (l[i]+u[i])/2 for i in range(n)]

bounds = [-5.12]*10, [5.12]*10
alpha = 0.01

# Caso 1: 10D->1D
def f1D(z):
    x = cosdense(z[0], bounds[0], bounds[1], alpha)
    return rastrigin(x)

# Caso 2: 10D->2D->1D
def f2D_1D(z):
    z1 = (np.pi/2)*(1-np.cos(z[0]))
    z2 = (np.pi/2)*(1-np.cos(1.5*z[0]))
    x1 = cosdense(z1, bounds[0][:5], bounds[1][:5], alpha)
    x2 = cosdense(z2, bounds[0][5:], bounds[1][5:], alpha)
    return rastrigin(x1 + x2)

# Caso 3: 10D->5D->2D->1D
def f1D_final(z):
    u = (np.pi/2)*(1-np.cos(z[0]))
    v = (np.pi/2)*(1-np.cos(1.5*z[0]))
    z_vec = [(np.pi/2)*(1-np.cos(u)), (np.pi/2)*(1-np.cos(v))]
    x = []
    for i in range(2):
        x += cosdense(z_vec[i], bounds[0][i*5:(i+1)*5], bounds[1][i*5:(i+1)*5], alpha)
    return rastrigin(x)

# Otimização com DIRECT
res1 = direct(f1D, bounds=[(0,np.pi)])
res2 = direct(f2D_1D, bounds=[(0,np.pi)])
res3 = direct(f1D_final, bounds=[(0,np.pi)])

print('\nResultados usando DIRECT:\n')
print(f'Caso 10D->1D: valor={res1.fun:.5f}, avaliações={res1.nfev}')
print(f'Caso 10D->2D->1D: valor={res2.fun:.5f}, avaliações={res2.nfev}')
print(f'Caso 10D->5D->2D->1D: valor={res3.fun:.5f}, avaliações={res3.nfev}')


Resultados usando DIRECT:

Caso 10D->1D: valor=104.30140, avaliações=177
Caso 10D->2D->1D: valor=141.45376, avaliações=303
Caso 10D->5D->2D->1D: valor=121.54792, avaliações=201


In [1]:
import numpy as np
from scipy.optimize import direct

# Função Rastrigin em 10 dimensões
def rastrigin(x):
    A = 10
    return A * len(x) + sum([(xi**2 - A * np.cos(2 * np.pi * xi)) for xi in x])

# Curva Cosdense com alpha variável
def cosdense(z, l, u, alpha):
    n = len(l)
    beta = [1] + [(np.pi/alpha)**i * np.prod([u[j]-l[j] for j in range(i)]) for i in range(1,n)]
    return [(l[i]-u[i])/2 * np.cos(beta[i]*z) + (l[i]+u[i])/2 for i in range(n)]

bounds = [-5.12]*10, [5.12]*10

alpha_list = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0]

# Caso 10D->1D
print('Resultados caso 10D->1D:\n')
for alpha in alpha_list:
    def f1D(z):
        x = cosdense(z[0], bounds[0], bounds[1], alpha)
        return rastrigin(x)
    result = direct(f1D, bounds=[(0,np.pi)])
    print(f'alpha={alpha}, valor={result.fun:.5f}, avaliações={result.nfev}')

# Caso 10D->2D->1D
print('\nResultados caso 10D->2D->1D:\n')
for alpha in alpha_list:
    def f2D_1D(z):
        z1 = (np.pi/2)*(1-np.cos(z[0]))
        z2 = (np.pi/2)*(1-np.cos(1.5*z[0]))
        x1 = cosdense(z1, bounds[0][:5], bounds[1][:5], alpha)
        x2 = cosdense(z2, bounds[0][5:], bounds[1][5:], alpha)
        return rastrigin(x1 + x2)
    result = direct(f2D_1D, bounds=[(0,np.pi)])
    print(f'alpha={alpha}, valor={result.fun:.5f}, avaliações={result.nfev}')

# Caso 10D->5D->2D->1D
print('\nResultados caso 10D->5D->2D->1D:\n')
for alpha in alpha_list:
    def f1D_final(z):
        u = (np.pi/2)*(1-np.cos(z[0]))
        v = (np.pi/2)*(1-np.cos(1.5*z[0]))
        z_vec = [(np.pi/2)*(1-np.cos(u)), (np.pi/2)*(1-np.cos(v))]
        x = []
        for i in range(2):
            x += cosdense(z_vec[i], bounds[0][i*5:(i+1)*5], bounds[1][i*5:(i+1)*5], alpha)
        return rastrigin(x)
    result = direct(f1D_final, bounds=[(0,np.pi)])
    print(f'alpha={alpha}, valor={result.fun:.5f}, avaliações={result.nfev}')


Resultados caso 10D->1D:

alpha=0.001, valor=118.32144, avaliações=153
alpha=0.005, valor=115.00384, avaliações=219
alpha=0.01, valor=104.30140, avaliações=177
alpha=0.05, valor=118.45059, avaliações=281
alpha=0.1, valor=138.10530, avaliações=289
alpha=0.5, valor=111.23308, avaliações=175
alpha=1.0, valor=104.45098, avaliações=209

Resultados caso 10D->2D->1D:

alpha=0.001, valor=127.44617, avaliações=207
alpha=0.005, valor=130.34204, avaliações=223
alpha=0.01, valor=141.45376, avaliações=303
alpha=0.05, valor=125.95577, avaliações=227
alpha=0.1, valor=140.21436, avaliações=237
alpha=0.5, valor=72.60624, avaliações=127
alpha=1.0, valor=82.35604, avaliações=199

Resultados caso 10D->5D->2D->1D:

alpha=0.001, valor=149.91495, avaliações=309
alpha=0.005, valor=113.80837, avaliações=271
alpha=0.01, valor=121.54792, avaliações=201
alpha=0.05, valor=140.51905, avaliações=235
alpha=0.1, valor=138.71722, avaliações=219
alpha=0.5, valor=110.01114, avaliações=227
alpha=1.0, valor=122.46218, aval

In [5]:
import numpy as np
!pip install hilbertcurve
from scipy.optimize import differential_evolution
from hilbertcurve.hilbertcurve import HilbertCurve

# Função Rastrigin
def rastrigin(x):
    A = 10
    return A * len(x) + sum([(xi**2 - A * np.cos(2 * np.pi * xi)) for xi in x])

bounds = [-5.12]*10, [5.12]*10

# Implementação correta da curva de Peano através da curva de Hilbert
def peano_curve(t, dim, bounds):
    p = 10  # número de bits
    max_h = 2**(p*dim)-1
    t_scaled = int(t * max_h)
    hc = HilbertCurve(p, dim)
    coords = hc.point_from_distance(t_scaled)
    return [bounds[0][i] + (bounds[1][i]-bounds[0][i])*coords[i]/(2**p-1) for i in range(dim)]

# Caso 10D -> 1D
def f_peano_1D(z):
    x = peano_curve(z[0]/np.pi, 10, bounds)
    return rastrigin(x)

# Caso 10D -> 2D -> 1D
def f_peano_2D_1D(z):
    z1 = (1-np.cos(z[0]))/2
    z2 = (1-np.cos(1.5*z[0]))/2
    x1 = peano_curve(z1, 5, (bounds[0][:5], bounds[1][:5]))
    x2 = peano_curve(z2, 5, (bounds[0][5:], bounds[1][5:]))
    return rastrigin(x1 + x2)

# Caso 10D -> 5D -> 2D -> 1D
def f_peano_5D_2D_1D(z):
    u = (1-np.cos(z[0]))/2
    v = (1-np.cos(1.5*z[0]))/2
    z_vec = [(1-np.cos(u))/2, (1-np.cos(v))/2]
    x = []
    for i in range(2):
        x += peano_curve(z_vec[i], 5, (bounds[0][i*5:(i+1)*5], bounds[1][i*5:(i+1)*5]))
    return rastrigin(x)

# Otimização usando Differential Evolution
res_peano_1D = differential_evolution(f_peano_1D, bounds=[(0,np.pi)])
res_peano_2D_1D = differential_evolution(f_peano_2D_1D, bounds=[(0,np.pi)])
res_peano_5D_2D_1D = differential_evolution(f_peano_5D_2D_1D, bounds=[(0,np.pi)])

# Impressão dos resultados
print('Resultados usando Curva de Peano (via Hilbert) com Differential Evolution:\n')
print(f'Caso 10D->1D: valor={res_peano_1D.fun:.5f}, avaliações={res_peano_1D.nfev}')
print(f'Caso 10D->2D->1D: valor={res_peano_2D_1D.fun:.5f}, avaliações={res_peano_2D_1D.nfev}')
print(f'Caso 10D->5D->2D->1D: valor={res_peano_5D_2D_1D.fun:.5f}, avaliações={res_peano_5D_2D_1D.nfev}')


Resultados usando Curva de Peano (via Hilbert) com Differential Evolution:

Caso 10D->1D: valor=67.26969, avaliações=15057
Caso 10D->2D->1D: valor=69.53657, avaliações=2191
Caso 10D->5D->2D->1D: valor=83.80456, avaliações=1047


In [6]:
import numpy as np
!pip install hilbertcurve
from scipy.optimize import differential_evolution
from hilbertcurve.hilbertcurve import HilbertCurve

# Função Rastrigin
def rastrigin(x):
    A = 10
    return A * len(x) + sum([(xi**2 - A * np.cos(2 * np.pi * xi)) for xi in x])

bounds = [-5.12]*10, [5.12]*10

# Implementação direta da curva de Hilbert
def hilbert_curve(t, dim, bounds):
    p = 10  # número de bits
    max_h = 2**(p*dim)-1
    t_scaled = int(t * max_h)
    hc = HilbertCurve(p, dim)
    coords = hc.point_from_distance(t_scaled)
    return [bounds[0][i] + (bounds[1][i]-bounds[0][i])*coords[i]/(2**p-1) for i in range(dim)]

# Caso 10D -> 1D
def f_hilbert_1D(z):
    x = hilbert_curve(z[0]/np.pi, 10, bounds)
    return rastrigin(x)

# Caso 10D -> 2D -> 1D
def f_hilbert_2D_1D(z):
    z1 = (1-np.cos(z[0]))/2
    z2 = (1-np.cos(1.5*z[0]))/2
    x1 = hilbert_curve(z1, 5, (bounds[0][:5], bounds[1][:5]))
    x2 = hilbert_curve(z2, 5, (bounds[0][5:], bounds[1][5:]))
    return rastrigin(x1 + x2)

# Caso 10D -> 5D -> 2D -> 1D
def f_hilbert_5D_2D_1D(z):
    u = (1-np.cos(z[0]))/2
    v = (1-np.cos(1.5*z[0]))/2
    z_vec = [(1-np.cos(u))/2, (1-np.cos(v))/2]
    x = []
    for i in range(2):
        x += hilbert_curve(z_vec[i], 5, (bounds[0][i*5:(i+1)*5], bounds[1][i*5:(i+1)*5]))
    return rastrigin(x)

# Otimização usando Differential Evolution
res_hilbert_1D = differential_evolution(f_hilbert_1D, bounds=[(0,np.pi)])
res_hilbert_2D_1D = differential_evolution(f_hilbert_2D_1D, bounds=[(0,np.pi)])
res_hilbert_5D_2D_1D = differential_evolution(f_hilbert_5D_2D_1D, bounds=[(0,np.pi)])

# Impressão dos resultados
print('Resultados usando Curva de Hilbert com Differential Evolution:\n')
print(f'Caso 10D->1D: valor={res_hilbert_1D.fun:.5f}, avaliações={res_hilbert_1D.nfev}')
print(f'Caso 10D->2D->1D: valor={res_hilbert_2D_1D.fun:.5f}, avaliações={res_hilbert_2D_1D.nfev}')
print(f'Caso 10D->5D->2D->1D: valor={res_hilbert_5D_2D_1D.fun:.5f}, avaliações={res_hilbert_5D_2D_1D.nfev}')

Resultados usando Curva de Hilbert com Differential Evolution:

Caso 10D->1D: valor=82.15564, avaliações=477
Caso 10D->2D->1D: valor=79.88833, avaliações=311
Caso 10D->5D->2D->1D: valor=99.63595, avaliações=312


In [10]:
import numpy as np
from scipy.optimize import differential_evolution

# Função Rastrigin
def rastrigin(x):
    A = 10
    return A * len(x) + sum([(xi**2 - A * np.cos(2 * np.pi * xi)) for xi in x])

bounds = [-5.12]*10, [5.12]*10

# Implementação direta da curva de Morton (Z-order)
def morton_curve(t, dim, bounds):
    bits = 10
    max_coord = 2**bits - 1
    t_scaled = int(t * (2**(bits*dim)-1))
    coords = [0]*dim
    for i in range(bits):
        for d in range(dim):
            coords[d] |= ((t_scaled >> (i*dim + d)) & 1) << i
    return [bounds[0][i] + (bounds[1][i]-bounds[0][i])*coords[i]/max_coord for i in range(dim)]

# Caso 10D -> 1D
def f_morton_1D(z):
    x = morton_curve(z[0]/np.pi, 10, bounds)
    return rastrigin(x)

# Caso 10D -> 2D -> 1D
def f_morton_2D_1D(z):
    z1 = (1-np.cos(z[0]))/2
    z2 = (1-np.cos(1.5*z[0]))/2
    x1 = morton_curve(z1, 5, (bounds[0][:5], bounds[1][:5]))
    x2 = morton_curve(z2, 5, (bounds[0][5:], bounds[1][5:]))
    return rastrigin(x1 + x2)

# Caso 10D -> 5D -> 2D -> 1D
def f_morton_5D_2D_1D(z):
    u = (1-np.cos(z[0]))/2
    v = (1-np.cos(1.5*z[0]))/2
    z_vec = [(1-np.cos(u))/2, (1-np.cos(v))/2]
    x = []
    for i in range(2):
        x += morton_curve(z_vec[i], 5, (bounds[0][i*5:(i+1)*5], bounds[1][i*5:(i+1)*5]))
    return rastrigin(x)

# Otimização usando Differential Evolution
res_morton_1D = differential_evolution(f_morton_1D, bounds=[(0,np.pi)])
res_morton_2D_1D = differential_evolution(f_morton_2D_1D, bounds=[(0,np.pi)])
res_morton_5D_2D_1D = differential_evolution(f_morton_5D_2D_1D, bounds=[(0,np.pi)])

# Impressão dos resultados
print('Resultados usando Curva de Morton (Z-order) com Differential Evolution:\n')
print(f'Caso 10D->1D: valor={res_morton_1D.fun:.5f}, avaliações={res_morton_1D.nfev}')
print(f'Caso 10D->2D->1D: valor={res_morton_2D_1D.fun:.5f}, avaliações={res_morton_2D_1D.nfev}')
print(f'Caso 10D->5D->2D->1D: valor={res_morton_5D_2D_1D.fun:.5f}, avaliações={res_morton_5D_2D_1D.nfev}')

Resultados usando Curva de Morton (Z-order) com Differential Evolution:

Caso 10D->1D: valor=83.43822, avaliações=4317
Caso 10D->2D->1D: valor=77.47391, avaliações=2232
Caso 10D->5D->2D->1D: valor=47.68404, avaliações=1272
